In [29]:
#!/usr/bin/python
import psycopg2
import json
import requests
from datetime import datetime
from configparser import ConfigParser
from contextlib import contextmanager
from requests_oauthlib import OAuth1

@contextmanager
def newStock(typeStored):
    try:
        params = dbConfig()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        yield cur
    finally:
        conn.commit()
        print(typeStored+' Stored')

def isNumber(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def dbValues(str):
    y = list(str.split(","))
    y = list(map(lambda ele: float(ele) if isNumber(ele) else ele, y))
    return y

def dbConfig(filename='database.ini', section='postgresql'):
    # create parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    db = {}

    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in {1}'.format(section, filename))

    return db

def oaConfig(filename='oauth.ini', section='Ally'):
    # create parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    oa = {}

    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            oa[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in {1}'.format(section, filename))

    return oa

def oaConn():
    # Client / Application Credentials
    params = oaConfig()
    # Default variables
    client_key = params['client_key']
    client_secret = params['client_secret']
    resource_owner_key = params['resource_owner_key']
    resource_owner_secret = params['resource_owner_secret']
    # OAuth Connection
    headeroauth = OAuth1(client_key, client_secret,
                     resource_owner_key, resource_owner_secret,
                     signature_method='PLAINTEXT')
    return headeroauth

def getBankData( token_url, headeroauth ):
    # Get data based on OAuth token and URL
    r = requests.get(token_url, auth=headeroauth)
    # Transform returned JSON data
    data = json.loads(r.text)
    rec = data["response"]["quotes"]["quote"]
    return rec

def stockData( headeroauth ):
    # Stocks Symbols
    qry = "AMZN,MSFT,AAPL,BLDP,CERC,OXY,BAC,CSCO,AMD,KO,WORK,LVGO,HIX,V,T,TWTR,KSS,SYY,MU,SPY,XLK,W"
    token_url = "https://api.tradeking.com/v1/market/ext/quotes.json?symbols="+qry
    # Get Bank data
    rec = getBankData( token_url, headeroauth )
    # Use Iterator to open, insert, and commit
    with newStock('Stock') as stk:
        for i in rec:
            x = i["symbol"] +', '+ i["bid"] +', '+ i["ask"]+ ', ' + i["adp_100"] + ', ' + i["adp_200"] + ', ' + i["adp_50"] + ', ' + i["adv_21"] + ', ' + i["cl"] + ', ' + i["datetime"] + ', ' + i["last"] + ', ' + i["lo"] + ', ' + i["hi"] + ', ' + i["wk52hi"] + ', ' + i["wk52lo"]
            # Format according to Postgres sintax
            pgValue = dbValues(x)
            stk.execute("INSERT INTO f_stockmarket ( symbol, bid, ask, adp_100, adp_200, adp_50, adv_21, cl, datetime, last, lo, hi, wk52hi, wk52lo) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", pgValue)

def optionData( headeroauth ):
    #Options Symbols
    qry = "AMZN200828C03180000,AMZN200828P03180000,MSFT200828C00205000,MSFT200828P00205000,AAPL200828C00420000,AAPL200828P00420000,T200828C00029500,T200828P00029500"
    token_url = "https://api.tradeking.com/v1/market/ext/quotes.json?symbols="+qry
    # Get Bank data
    rec = getBankData( token_url, headeroauth )
    # Use Iterator to open, insert, and commit
    with newStock('Option') as stk:
        for i in rec:
            x = i["symbol"] +', '+ i["bid"] +', '+ i["ask"]+ ', ' + i["idelta"] + ', ' + i["igamma"] + ', ' + i["imp_volatility"] + ', ' + i["opn"] + ', ' + i["cl"] + ', ' + i["datetime"] + ', ' + i["last"] + ', ' + i["lo"] + ', ' + i["hi"] + ', ' + i["put_call"] + ', ' + i["rootsymbol"] + ', ' + i["strikeprice"]  + ', ' + i["timestamp"]
            pgValue = dbValues(x)
            stk.execute("INSERT INTO f_optionmarket ( symbol, bid, ask, idelta, igamma, imp_volatility, opn, cl, datetime, last, lo, hi, put_call, rootsymbol, strikeprice, timestamp) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",pgValue)


In [30]:
def main():
    oaHeader = oaConn()
    stockData( oaHeader )
    optionData( oaHeader )

In [31]:
if __name__ == '__main__': main()

Stock Stored
Option Stored
